Import dependencies

In [2]:
import numpy as np 
import pandas as pd 
import csv
#from sklearn.preprocessing import Imputer

Import the file containing the location data of all the sensors (cycles + cars). The file aurrently has only the location data of sensors counting cars.

In [2]:
location_data = pd.read_csv('tellepunkter.csv',sep=',')
#Look into the format of the imported data
location_data.head()
OSLOLAT1=6642806
OSLOLONG1=597912

location_data['DTC']=(((location_data['LATITUDE']-OSLOLAT1)**2 +(location_data['LONGITUDE']-OSLOLONG1)**2)**0.5)/1000

print(len(location_data['DTC']))
location_data=location_data.dropna()
#print(len(location_data['DTC']))
r1=location_data['DTC'].max()
r2=location_data['DTC'].min()
range1=r1-r2
setlim=0.04*range1;
#print(setlim,r1,r2)
location_data['SENSOR_TYPE']
#Find row index with nan 
#print(location_data.isnull().any(axis=0))
#inds = location_data.isnull().any(1).nonzero()[0]
#print(location_data.loc[inds[40:],'DTC'])

#Find index of sensor with sykkel

idx = location_data.index[location_data['SENSOR_TYPE']== 'Sykkel'].tolist()
#print(location_data.loc[idx])
#print(idx)
#abc=location_data.loc[idx,'SENSOR_ID']
#print(abc)
#print(location_data[idx,'SENSOR_ID'])

155


Import the car, cycle and met data

In [3]:
car_data = pd.read_csv('car_feb2015_feb2016.csv',sep=';')
print("Rows in car data ",len(car_data))
#print(car_data.isnull().sum(axis=0).sum())
#print(cycle_data.count().sum())
cs=car_data.size
csmissing=car_data.isnull().sum(axis=0).sum()
print('carpercentmissing',csmissing)
print('carpercentmissing',(csmissing/cs))
car_data=car_data.drop(['AVGSPEED '],axis=1)
car_data.head()

Rows in car data  101669
carpercentmissing 0
carpercentmissing 0.0


,SENSOR_ID,YEAR,MONTH,DAY,STARTHOUR,DIRECTION,RESOLUTION,SUM
0,300016,2015,2,1,0,1,HOURLY,381
1,300016,2015,2,1,1,1,HOURLY,284
2,300016,2015,2,1,2,1,HOURLY,165
3,300016,2015,2,1,3,1,HOURLY,98
4,300016,2015,2,1,4,1,HOURLY,87


In [4]:
#for sensor_id in list(car_data['SENSOR_ID'].unique()):
#    print(sensor_id)
#Only retain the relevant sensor data location

#location_data = location_data[(location_data.SENSOR_ID==300016)| (location_data.SENSOR_ID==300083)| 
#                              (location_data.SENSOR_ID==300099)| (location_data.SENSOR_ID==300233)| 
#                              (location_data.SENSOR_ID==302257)]


In [5]:
cycle_data = pd.read_csv('cycle_feb2015_feb2016.csv',sep=',')
print("Rows in cycle data ",len(cycle_data))
print(cycle_data.isnull().sum(axis=0).sum())
#print(cycle_data.count().sum())
print(cycle_data.size)
print('cyclepercentmissing',cycle_data.isnull().sum(axis=0).sum()/cycle_data.size)

cycle_data.head()


Rows in cycle data  868084
504602
4340420
cyclepercentmissing 0.116256491307


,SENSOR_ID,TIME,LANE,SUM,RESOLUTION
0,10000010,2015-02-01T00:00:00,99.0,5.0,HOURLY
1,10000010,2015-02-01T01:00:00,99.0,2.0,HOURLY
2,10000010,2015-02-01T02:00:00,99.0,6.0,HOURLY
3,10000010,2015-02-01T03:00:00,99.0,5.0,HOURLY
4,10000010,2015-02-01T04:00:00,99.0,4.0,HOURLY


In [6]:
weather_data = pd.read_csv('Oslo_Blindern_Weather1.txt',sep=';')
print("Rows in the met data ",len(weather_data))
weather_data.head()

Rows in the met data  8784


,St.no,Year,Mnth,Date,Time,DirectionDeg,Speedmps,TempC,Pptmm,Humidpercen,Snowdepthcm
0,18700,2015,2,1,1,7,5.8,-1.0,0.0,75,
1,18700,2015,2,1,2,6,6.2,-1.3,0.0,77,
2,18700,2015,2,1,3,12,6.5,-1.5,0.1,79,
3,18700,2015,2,1,4,7,5.9,-1.5,0.0,76,
4,18700,2015,2,1,5,5,6.3,-1.7,0.0,77,


In [7]:
#Looking at the data we realized that location of only 74 sensors is available. We convert the coordinates of these
#sensors from UTM to lat-lon for plotting their location later in this notebook.
latitude_UTM=location_data['LATITUDE'].values[0:74]
longitude_UTM=location_data['LONGITUDE'].values[0:74]
lats = []
lons = []
import utm
for i in range(len(latitude_UTM)):
    temp_lats,temp_lons=utm.to_latlon(longitude_UTM[i], latitude_UTM[i], 33, 'U')
    lats.append(temp_lats),lons.append(temp_lons)
labels=location_data['SENSOR_ID'].values[0:74]

northerly=[59.55,59.52,59.57,59.55]
easterly=[10.45,10.47,10.45,10.44]

utm1 = []
utm2 = []
utmcord=np.zeros(len(northerly))
for i in range(len(northerly)):
    utmcoords=utm.from_latlon(northerly[i],easterly[i])
    utm1.append(utmcoords[0]),utm2.append(utmcoords[1])
    utmcord[i]=(((utmcoords[1]-OSLOLAT1)**2 +(utmcoords[0]-OSLOLONG1)**2)**0.5)/1000
#print(utm1,utm2)
#print(utmcord)
index = [i for i in range(1, len(northerly)+1)]
UTMadd=pd.DataFrame(utmcord,index,['DTC'])
#print(cycle_data.SENSOR_ID.unique())
UTMadd['SENSOR_ID']= [10000010,10000210,10000480,10000790]

#print(UTMadd)

In [8]:
sum1=0
for sensor_id in list(cycle_data['SENSOR_ID'].unique()):
    missing=cycle_data[cycle_data.SENSOR_ID==sensor_id].isnull().values.any()
    if missing==False:
        print("Sensor id ",sensor_id, location_data[location_data.SENSOR_ID==sensor_id].SENSOR_NAVN.values,
              "Rows ",len(cycle_data[cycle_data.SENSOR_ID==sensor_id]))
        sum1=sum1 + len(cycle_data[cycle_data.SENSOR_ID==sensor_id])
#print(cycle_data.isnull().sum(axis=0).sum())
#print(cycle_data.count().sum())
#print('Cycle data size',cycle_data.size)
print('cyclepercentmissing',cycle_data.isnull().sum(axis=0).sum()/cycle_data.size)

SyntaxError: EOL while scanning string literal (<ipython-input-8-a32a12919595>, line 8)

I then used google earth to extract the coordinates of these locations.
 * 10000010 (59°55'59.53"N,  10°45'27.53"E)
 * 10000210 (59°52'33.68"N,  10°47'21.58"E)
 * 10000480 (59°57'17.15"N,  10°45'40.28"E) 
 * 10000790 (59°55'14.14"N,  10°44'31.83"E)

Let us plot these cordinates

In [ ]:
lats.append(59+55/60+14/3600); lons.append(10+44/60+31/3600)
lats.append(59+57/60+17/3600); lons.append(10+45/60+40/3600)
lats.append(59+52/60+33/3600); lons.append(10+47/60+21/3600)
lats.append(59+55/60+59/3600); lons.append(10+45/60+27/3600)

labels=np.append(labels,'Ullevalsvn 19')
labels=np.append(labels,'Maridalsvn')
labels=np.append(labels,'Ekebergvn')
labels=np.append(labels,'Akerselva')

#from mpl_toolkits.basemap import Basemap
#import matplotlib.pyplot as plt
#map = Basemap(projection='merc',resolution = 'h', area_thresh = 0.1,
#              llcrnrlon=10.6, llcrnrlat=59.8,urcrnrlon=11, urcrnrlat=60)
#plt.figure(figsize=(16,16))
#map.drawcoastlines(linewidth=0.2,color='grey')
#map.drawcountries(linewidth=0.2,color='grey')
#map.drawmapboundary()
##map.shadedrelief()
#map.bluemarble()
#x,y = map(lons, lats)
#map.plot(x, y, 'bo', markersize=7)
#x_offsets = 20*np.ones_like(lats)
#y_offsets = 400*np.ones_like(lats)
#for label, xpt, ypt, x_offset, y_offset in zip(labels, x, y, x_offsets, y_offsets):
#    plt.text(xpt+x_offset, ypt+y_offset, label)

##plt.savefig('bessaker.pdf',bbox_inches='tight')
#plt.show()


Clean and format the data

In [ ]:
cycle_data['TIME'] =  pd.to_datetime(cycle_data['TIME'], format='%Y-%m-%dT%H:%M:%S.%f')
cycle_data = cycle_data.drop(["RESOLUTION"],axis=1)
cycle_sensor_location = cycle_data.merge(location_data,on=['SENSOR_ID'])
#print(cycle_data.SENSOR_ID.unique())
cycle_data=cycle_data.merge(UTMadd,on=['SENSOR_ID']) 
print(cycle_data.head())
#cycle_sensor_location['DTCcomp'] = np.where(cycle_sensor_location['DTC']-car_sensor_location['DTC'] <= setlim, 1,0)

In [ ]:
df2=pd.DataFrame(weather_data[['Year','Mnth','Date','Time']]).rename(columns={'Year': 'year', 'Mnth': 'month', 'Date':'day','Time':'Hour'})
df2.head()
weather_data['TIME']=pd.to_datetime(df2)
weather_data=weather_data.drop(["Year","Mnth","Date","Time","Snowdepthcm","DirectionDeg"],axis=1)

In [ ]:
df3=pd.DataFrame(car_data[['YEAR','MONTH','DAY','STARTHOUR']]).rename(columns={'YEAR': 'year', 'MONTH': 'month', 'DAY':'day','STARTHOUR':'Hour'})
car_data['TIME']=pd.to_datetime(df3)
car_data=car_data.drop(["YEAR","MONTH","DAY","STARTHOUR","RESOLUTION","RESOLUTION"],axis=1)

car_sensor_location = car_data.merge(location_data,on=['SENSOR_ID'])






In [ ]:
weather_data.rename(columns={'Speedmps':'WIND_MAG','Pptmm':
                             'PPT','Humidpercen':'RH','TempC':'TEMP',
                             'Snowdepthcm':'SNOW_DEPTH','St.no':'STATION_ID'}, inplace=True)

Merge the car anc cycle data with the weather data

In [ ]:
#print("Rows in car data",len(car_data))
#print("Rows in cycling ",len(cycle_data))
#car_met_cycle1 = cycle_data.merge(car_data,on=['TIME,SENSOR_ID'])
#print("Rows in cycling data combined with met ",len(car_met_cycle1))
#car_met_cycle1 = pd.merge(car_data, cycle_data,on=['TIME','SENSOR_ID'])
#print("Rows in cycling data combined with met ",len(car_met_cycle1) )
#car_met_sensor.head()

In [ ]:
cycle_met = cycle_data.merge(weather_data,on=['TIME'])


In [ ]:
car_met = car_data.merge(weather_data,on=['TIME'])

In [ ]:
#car_met_cycle1_met=car_met_cycle1.merge(weather_data,on=['TIME'])

In [ ]:
#print("Rows in car data combined cycle data with met ",len(car_met_cycle1_met))
print("Rows in cycling data combined with met ",len(cycle_met))
print("Rows in car data combined with met ",len(car_met))
#print("Rows in car data combined cycle data with met ",car_met_cycle1_met.shape)

In [ ]:
import calendar
#cycle_met['WEEKDAY'] = cycle_met['TIME'].dt.weekday_name #Insert a column with weekday name
cycle_met['WEEKDAY'] = cycle_met['TIME'].dt.day_name
cycle_met['MONTH'] = cycle_met['TIME'].dt.month #Insert a column with month number
cycle_met['MONTH'] = cycle_met['MONTH'].apply(lambda x: calendar.month_abbr[x]) #Conver month no. to month name
cycle_met['HOUR'] = cycle_met['TIME'].dt.hour #Insert a column with hour

#car_met['WEEKDAY'] = car_met['TIME'].dt.weekday_name
car_met['WEEKDAY'] = car_met['TIME'].dt.day_name
car_met['MONTH'] = car_met['TIME'].dt.month
car_met['MONTH'] = car_met['MONTH'].apply(lambda x: calendar.month_abbr[x])
car_met['HOUR'] = car_met['TIME'].dt.hour

In [ ]:
for sensor_id in list(car_met['SENSOR_ID'].unique()):
    missing=car_met[car_met.SENSOR_ID==sensor_id].isnull().values.any()
    if missing==False:
        print("Station id ",sensor_id)
        

The above are the only stations where no data is missing. let us retain only these sensor data.

In [ ]:
cycle_met_clean_data = cycle_met[(cycle_met.SENSOR_ID==10000210)| (cycle_met.SENSOR_ID==10000480) | (cycle_met.SENSOR_ID==10000790)]
car_met_clean_data = car_met[(car_met.SENSOR_ID==300016)| (car_met.SENSOR_ID==300099) | (car_met.SENSOR_ID==300233) | (car_met.SENSOR_ID==302257
)| (car_met.SENSOR_ID==300083)]

Write them to csv files

In [ ]:
cycle_met_clean_data.to_csv('cycle_met.csv',sep=';',index=False)
car_met_clean_data.to_csv('car_met.csv',sep=';',index=False)
cycle_met_clean_sensor_location = cycle_met_clean_data.merge(location_data,on=['SENSOR_ID'])
car_met_clean_sensor_location = car_met_clean_data.merge(location_data,on=['SENSOR_ID'])

In [ ]:
print("Rows in cycling data combined with met ",len(cycle_met_clean_data),len(car_met_clean_data),len(cycle_met_clean_sensor_location),len(car_met_clean_sensor_location))
#print("Rows in cycling data combined with met ",)


In [ ]:
#car_met.head()

In [ ]:
#cycle_met_clean_data.head()

In [ ]:
car_met_clean_sensor_location=car_met_clean_sensor_location.drop(["SENSOR_NAVN","LATITUDE","LONGITUDE","STREKNING_NAVN","FORMAT","SENSOR_STATUS","SENSOR_TYPE","OPPLOSNING"],axis=1)
#car_met_clean_sensor_location.head()

In [ ]:
#cycle_sensor_location = cycle_met_clean_data.merge(location_data,on=['SENSOR_ID'])
#car_sensor_location = car_met_clean_data.merge(location_data,on=['SENSOR_ID'])
#car_met_cycle = cycle_met.merge(car_met,on=['TIME'],how='right')


#cycle_sensor_location=cycle_sensor_location.drop(["SENSOR_NAVN","LATITUDE","LONGITUDE","STREKNING_NAVN","FORMAT","SENSOR_STATUS","SENSOR_TYPE","OPPLOSNING"],axis=1)
#cycle_sensor_location.head()


In [ ]:
#print("Rows in cycling data combined with car on time ",len(car_met_cycle))
#car_met_cycle.head()
#car_sensor_location=car_sensor_location.drop(["SENSOR_NAVN","LATITUDE","LONGITUDE","STREKNING_NAVN","FORMAT","SENSOR_STATUS","SENSOR_TYPE","OPPLOSNING"],axis=1)
#car_sensor_location.head()

In [ ]:
#print(car_sensor_location.shape)
#print(cycle_sensor_location.shape)
#print(cycle_sensor_location.isnull().sum())
#print(car_sensor_location.isnull().sum())
#print('location',location_data.isnull().sum())
#location_data.index[location_data['SENSOR_ID'] == 10000210].tolist()
#print('length',len(cycle_met_clean_data.index[cycle_met_clean_data['SENSOR_ID'] == 10000210].tolist()))
#print(location_data.loc[location_data.index[location_data['SENSOR_ID'] == 10000210]])

In [ ]:
print(car_met_clean_sensor_location.shape)
car_met_clean_sensor_location.head()




In [ ]:
print(cycle_met_clean_data.shape)
cycle_met_clean_data.head()